In [1]:
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse

from models import AE, USAD
from utils import *

# AE(AutoEncoder) Reconstruct Test
- LSTM AE
- GRU AE
- TCN AE

In [2]:
# make dataset
df = pd.read_csv('./datasets/JNU_elec.csv').set_index('timestamp')
seq_data = create_seq(df, seq_len=24)

# split
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (26280, 24, 1)
Val: (8760, 24, 1)
Test: (3601, 24, 1)


In [3]:
# Min-Max normalize
min_val, max_val = np.min(train), np.max(train)

train_norm = minmax(train, min_val, max_val)
val_norm = minmax(val, min_val, max_val)
test_norm = minmax(test, min_val, max_val)

# make tensorflow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_norm, train_norm))
train_dataset = train_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_norm, val_norm))
val_dataset = val_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_norm, test_norm))
test_dataset = test_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [4]:
# LSTM AutoEncoder
lstmae = AE(input_dim=24, z_dim=64, d_hidden_dim=64, mode='LSTM')
# GRU AutoEncoder
gruae = AE(input_dim=24, z_dim=64, d_hidden_dim=64, mode='GRU')
# TCN
tcnae = AE(input_dim=24, z_dim=64, d_hidden_dim=64, e_hidden_dim=64, dilations=[1, 2, 4, 8, 16], mode='TCN')

In [ ]:
lstmhist = lstmae.fit(train_dataset, val_dataset)

epoch 1 train_loss: 0.0144 | 16.73 sec
epoch 1 val_loss: 0.0072 | 2.14 sec
epoch 2 train_loss: 0.0043 | 14.80 sec
epoch 2 val_loss: 0.0056 | 2.14 sec
epoch 3 train_loss: 0.0036 | 14.89 sec
epoch 3 val_loss: 0.0048 | 2.26 sec
epoch 4 train_loss: 0.0030 | 16.13 sec
epoch 4 val_loss: 0.0042 | 2.13 sec
epoch 5 train_loss: 0.0026 | 14.95 sec
epoch 5 val_loss: 0.0037 | 2.07 sec
epoch 6 train_loss: 0.0023 | 14.48 sec
epoch 6 val_loss: 0.0033 | 1.99 sec
epoch 7 train_loss: 0.0022 | 14.40 sec
epoch 7 val_loss: 0.0032 | 2.05 sec
epoch 8 train_loss: 0.0021 | 14.48 sec
epoch 8 val_loss: 0.0030 | 2.03 sec
epoch 9 train_loss: 0.0020 | 14.39 sec
epoch 9 val_loss: 0.0030 | 2.00 sec


In [ ]:
gruhist = gruae.fit(train_dataset, val_dataset)

In [ ]:
tcnhist = tcnae.fit(train_dataset, val_dataset)